In [1]:
import json
import pandas as pd
import numpy as np
import networkx as nx
import jellyfish
import os
import shutil
import subprocess
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering

In [2]:
def read_dataframe_from_file(filename, filesubstr):
    
    final_results = []
    initial_line = 2 - 1
    initial_line_result = 5 - 1
    
    num_class_f = open('Final_Results/' + filesubstr + '/num_class.txt')
    for line in num_class_f:
        num_class = line
        
    #print(num_class)

    f = open(filename, 'r')
    tmp_arr = []
    for i, line in enumerate(f):
        if i == initial_line:
            line = line.replace('MoJo', '')
            line = line.replace('.rsf', '')
            line = line.replace('(', '')
            line = line.replace(')', '')
            line = line.split('_')
            name = line[0].split('/')[0]
            version = line[0].split('-')[-1]
            n_cluster = line[1]
            affinity = line[2]
            linkage = line[3]
            #print(line)
            #print('Version', version)
            initial_line += 5
            tmp_arr.append(name)
            tmp_arr.append(version)
            tmp_arr.append(n_cluster)
            tmp_arr.append(affinity)
            tmp_arr.append(linkage)
        elif i == initial_line_result:
            n_MoJo = line.split(' ')[-1].split('\n')[0]
            
            #print(line)
            initial_line_result += 5
            tmp_arr.append(n_MoJo)
            tmp_arr.append((1 - (float(n_MoJo) / float(num_class))) * 100)
            final_results.append(tmp_arr)
            tmp_arr = []
            
    
    
    f.close()
    return final_results

In [ ]:
# Getting the smallest MoJo Value
rootdir = 'Final_Results/'
project_names = []

f = open('project_list.txt', 'r')
for line in f:
    print(line)
    project_names.append(line.split('\n')[0])

#project_names = ['apache-maven', 'apache-storm']
dir_arr = []

#Main clustering arr to be transformed to pandas dataframe later
cluster_main_arr = []

#Main CK arr to be transformed to pandas dataframe later
ck_sum_arr = []
ck_max_arr = []
ck_std_arr = []
ck_mean_arr = []

# Only considering the below ck_metrics
ck_metrics = ['cbo', 'wmc', 'dit', 'rfc', 'lcom',
       'totalMethods', 'staticMethods', 'publicMethods', 'privateMethods',
       'protectedMethods', 'defaultMethods', 'abstractMethods', 'finalMethods',
       'synchronizedMethods', 'totalFields', 'staticFields', 'publicFields',
       'privateFields', 'protectedFields', 'defaultFields', 'finalFields',
       'synchronizedFields', 'nosi', 'loc', 'returnQty', 'loopQty',
       'comparisonsQty', 'tryCatchQty', 'parenthesizedExpsQty',
       'stringLiteralsQty', 'numbersQty', 'assignmentsQty',
       'mathOperationsQty', 'variablesQty', 'maxNestedBlocks',
       'anonymousClassesQty', 'subClassesQty', 'lambdasQty', 'uniqueWordsQty',
       'modifiers']

depth = 3
for project_name in project_names:
    rootdir += project_name
    #print(rootdir)
    for root, dirs, files in os.walk(rootdir):
        for file in files:
            
            dir_arr.append(rootdir + '/' + str(file))
            
    for element in dir_arr:
        ver = str(element).replace('_class.csv', '')
        ver = ver.split('-')[-1]
        
        if '_class.csv' in element:
            
            df_sum = pd.read_csv(element)
            current_element_sum = df_sum.sum()
            
            df_max = pd.read_csv(element)
            current_element_max = df_max.max()
            
            df_std = pd.read_csv(element)
            current_element_std = df_std.std()
            
            df_mean = pd.read_csv(element)
            current_element_mean = df_mean.mean()
            
            tmp_sum = []
            tmp_max = []
            tmp_std = []
            tmp_mean = []
            
            tmp_sum.append(project_name)
            tmp_sum.append(ver)
            
            tmp_max.append(project_name)
            tmp_max.append(ver)
            
            tmp_std.append(project_name)
            tmp_std.append(ver)
            
            tmp_mean.append(project_name)
            tmp_mean.append(ver)
            
            for element in ck_metrics:
                tmp_sum.append(current_element_sum[element])
                tmp_max.append(current_element_max[element])
                tmp_std.append(current_element_std[element])
                tmp_mean.append(current_element_mean[element])
        
            ck_sum_arr.append(tmp_sum)
            ck_max_arr.append(tmp_max)
            ck_std_arr.append(tmp_std)
            ck_mean_arr.append(tmp_mean)
        
            
        elif '_results.txt' in element:
            #print(project_name, element)
            spark_df = pd.DataFrame(read_dataframe_from_file(element, project_name))
            spark_df.columns = ['name', 'version', 'n_clusters', 'affinity', 'linkage', 'n_MoJo', 'mojoFM']
            #spark_df['mojoFM'] = (1 - spark_df['n_MoJo'].astype(int)/spark_count)*100
            #print(spark_df[spark_df.n_MoJo == spark_df.n_MoJo.min()].iloc[0])
            #print(spark_df.loc[spark_df.groupby('version').mojoFM.idxmax()].reset_index(drop=True))
    for element in spark_df.loc[spark_df.groupby('version').mojoFM.idxmax()].reset_index(drop=True).iterrows():
        tmp_arr2 = []
        tmp_arr2.append(element[1]['name'])
        tmp_arr2.append(element[1]['version'])
        tmp_arr2.append(element[1]['n_clusters'])
        tmp_arr2.append(element[1]['affinity'])
        tmp_arr2.append(element[1]['linkage'])
        tmp_arr2.append(element[1]['n_MoJo'])
        tmp_arr2.append(element[1]['mojoFM'])
        
        cluster_main_arr.append(tmp_arr2)
    #cluster_main_arr.append(spark_df.loc[spark_df.groupby('version').mojoFM.idxmax()].reset_index(drop=True))
            #cluster_main_arr.append(spark_df[spark_df.n_MoJo == spark_df.n_MoJo.min()].iloc[0])
    
    rootdir = 'Final_Results/'
    #dir_arr = []
            

apache-storm

apache-cassandra

airbnb-lottie-android

apache-isis

apache-jmeter

apache-log4j

apache-maven

apache-spark

apache-tomcat

rzwitserloot-lombok

apache-tika



### For clustering

In [ ]:
final_clustering_result = pd.DataFrame(cluster_main_arr)
final_clustering_result.columns = ['name', 'version', 'n_clusters', 'affinity', 'linkage', 'n_MoJo', 'mojoFM']

In [ ]:
final_clustering_result.head()

In [ ]:
final_clustering_result['name'].unique()


In [ ]:
final_clustering_result.to_csv('clustering_result_10_ver.csv', index=False)

### For CK Metrics

In [ ]:
# For SUM CK Metric

ck_sum_df = pd.DataFrame(ck_sum_arr)
final_ck_columns = ['project_name', 'version']
for element in ck_metrics:
    final_ck_columns.append(element)
ck_sum_df.columns = final_ck_columns
ck_sum_df.head()

ck_sum_df.to_csv('ck_sum_10_ver.csv', index=False)

In [ ]:
# For Max CK Metric

ck_max_df = pd.DataFrame(ck_max_arr)
final_ck_columns = ['project_name', 'version']
for element in ck_metrics:
    final_ck_columns.append(element)
ck_max_df.columns = final_ck_columns
ck_max_df.head()

ck_max_df.to_csv('ck_max_10_ver.csv', index=False)

In [ ]:
# For SUM CK Metric

ck_std_arr = pd.DataFrame(ck_std_arr)
final_ck_columns = ['project_name', 'version']
for element in ck_metrics:
    final_ck_columns.append(element)
ck_std_arr.columns = final_ck_columns
ck_std_arr.head()

ck_std_arr.to_csv('ck_std_10_ver.csv', index=False)

In [ ]:
# For SUM CK Metric

ck_mean_arr = pd.DataFrame(ck_mean_arr)
final_ck_columns = ['project_name', 'version']
for element in ck_metrics:
    final_ck_columns.append(element)
ck_mean_arr.columns = final_ck_columns
ck_mean_arr.head()

ck_mean_arr.to_csv('ck_mean_10_ver.csv', index=False)

In [ ]:
test.project_name.unique()

In [ ]:
test.head(100)